# AUV 2D

In [1]:
import sys
sys.path.append("../src/dynamics")
from pontryagin import system
from sympy import init_printing
from sympy import *
init_printing()

In [2]:
# state
x, y, vx, vy = Matrix(symbols('x y v_x v_y', real=True))
state = Matrix([x, y, vx, vy])

# controls
ut, ux, uy = symbols('u_t', real=True, nonnegative=True), *symbols('u_x u_y', real=True, nonzero=True)
control = Matrix([ut, ux, uy])

# constants
CD, m, A, T, rho = symbols('C_D m A T rho', real=True, positive=True)

# state dynamics
dynamics = Matrix([
    vx,
    vy,
    T*ut*ux/m - Rational(1, 2)*rho*vx*sqrt(vx**2 + vy**2)*CD*A,
    T*ut*uy/m - Rational(1, 2)*rho*vy*sqrt(vx**2 + vy**2)*CD*A
])

# homotopy parameter
alpha = symbols('alpha', real=True, nonnegative=True)

# Lagrangian
lagrangian = alpha*ut + (1 - alpha)*ut**2

# equality constraints
equality = Matrix([sqrt(ux**2 + uy**2) - 1])

# inequality constraints
#inequality = Matrix([ut-1, -ut])

In [3]:
# instantiate the system
sys = system(state, control, dynamics, lagrangian, equality=equality, inequality=None)
sys.dfs

⎡                                       vₓ                                    
⎢                                                                             
⎢                                       v_y                                   
⎢                                                                             
⎢                                     ____________                            
⎢                                    ╱   2      2                             
⎢                       A⋅C_D⋅ρ⋅vₓ⋅╲╱  vₓ  + v_y     T⋅uₜ⋅uₓ                  
⎢                     - ────────────────────────── + ───────                  
⎢                                   2                   m                     
⎢                                                                             
⎢                                     ____________                            
⎢                                    ╱   2      2                             
⎢                      A⋅C_D⋅ρ⋅v_y⋅╲╱  vₓ  + v_y    

In [4]:
sys.KKTeqs

⎡                                                                             
⎢T⋅λᵥ ₓ⋅uₓ   T⋅λ_v_y⋅u_y                      T⋅λᵥ ₓ⋅uₜ        η₀⋅uₓ       T⋅λ
⎢───────── + ─────────── + α + 2⋅uₜ⋅(-α + 1), ───────── + ───────────────, ───
⎢    m            m                               m          ____________     
⎢                                                           ╱   2      2      
⎣                                                         ╲╱  uₓ  + u_y       

                              ____________    ⎤
_v_y⋅uₜ        η₀⋅u_y        ╱   2      2     ⎥
─────── + ───────────────, ╲╱  uₓ  + u_y   - 1⎥
 m           ____________                     ⎥
            ╱   2      2                      ⎥
          ╲╱  uₓ  + u_y                       ⎦

In [ ]:
sys.solve()